## Intelligent Student Query Advisor and FAQ Enhancer

### Project Outline :

* Project Introduction (Overview & Problem Statement)
* Data Description (Question & Database Information)
* Exploratory Data Analysis (EDA)
* Feature Engineering
* Machine Learning Models
* Model Evaluation
* Model Interpretability


### Overview :
This project is designed to build an end-to-end machine learning pipeline that efficiently processes student queries, categorizes them, predicts key service metrics, and uncovers hidden patterns in the data. By integrating both supervised and unsupervised learning techniques along with a robust database, the project aims to improve query resolution, enhance support services, and provide valuable insights for decision-makers

### Problem Statement :

Educational institutions often receive a high volume of student queries covering diverse topics such as academics, finances, and technical issues. Managing and responding to these queries efficiently is crucial for enhancing student satisfaction and administrative productivity. This project aims to develop a machine learning model to classify, predict, and analyze student queries while uncovering trends and improving decision-making processes—all within a structured Jupyter Notebook environment.

**Question :** The University of Excellence is looking to modernize its student support system. The administration wants to implement an automated system that processes student queries submitted via an online portal. Your organization has collected historical data comprising student questions, query categories, response times, satisfaction ratings, and resolution feedback. The aim is to design an end-to-end pipeline that will not only predict relevant answers based on past interactions but also analyze emerging trends from new queries.

You're provided with a CSV file capturing student queries along with metadata such as Query ID,Timestamp,Student ID,Query Text,Query Category,Satisfaction Score,Response Time,Resolved Flag.

In [1]:
# Python Library for Data Analysis In Machine Learning
import pandas as pd # panels for data manipulation and analysis
import numpy as np # numerical computing library